# Machine Learning GridSearch Pipeline

In [54]:
# Import libraries
import os
import sys

# cpu_count returns the number of CPUs in the system.
from multiprocessing import cpu_count

import numpy as np
import pandas as pd

# Import metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score

# Import preprocessing methods from sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures

# Import PCA
from sklearn.decomposition import PCA

# Import RFE
#from sklearn.feature_selection import RFE
#from sklearn.feature_selection import RFECV

# Import models from sklearn
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
#from sklearn.svm import SVC
#from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

# Import XGBClassifier
from xgboost.sklearn import XGBClassifier

# Import from sklearn
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
#from sklearn.pipeline import make_pipeline
from sklearn.externals import joblib
from sklearn.base import TransformerMixin
from sklearn.base import BaseEstimator

# Import plotting libraries
import matplotlib.pyplot as plt
import seaborn as sns

# Modify notebook settings
pd.options.display.max_columns = 150
pd.options.display.max_rows = 150
%matplotlib inline
plt.style.use('ggplot')

### Create paths to data file, append `src` directory to sys.path

In [2]:
# Create a variable for the project root directory
proj_root = os.path.join(os.pardir)

# Save path to the processed data file
# "dataset_processed.csv"
processed_data_file = os.path.join(proj_root,
                                   "data",
                                   "processed",
                                   "dataset_processed.csv")

In [3]:
# add the 'src' directory as one where we can import modules
src_dir = os.path.join(proj_root, "src")
sys.path.append(src_dir)

### Read in the processed data

In [4]:
# Read in the processed credit card client default data set.
df = pd.read_csv(processed_data_file, 
                           index_col=0)

df.head()

,y,bl_ratio_1,bl_ratio_2,bl_ratio_3,bl_ratio_4,bl_ratio_5,bl_ratio_6,blpl_ratio_1,blpl_ratio_2,blpl_ratio_3,blpl_ratio_4,blpl_ratio_5,blpl_ratio_6,limit_bal_log,age_log,bill_amt1_log,bill_amt2_log,bill_amt3_log,bill_amt4_log,bill_amt5_log,bill_amt6_log,pay_amt1_log,pay_amt2_log,pay_amt3_log,pay_amt4_log,pay_amt5_log,pay_amt6_log,sex_2,edu_2,edu_3,edu_4,marriage_1,marriage_2,marriage_3,pay_1_-2,pay_1_0,pay_1_1,pay_1_2,pay_1_3,pay_1_4,pay_1_5,pay_1_6,pay_1_7,pay_1_8,pay_2_-2,pay_2_0,pay_2_1,pay_2_2,pay_2_3,pay_2_4,pay_2_5,pay_2_6,pay_2_7,pay_2_8,pay_3_-2,pay_3_0,pay_3_1,pay_3_2,pay_3_3,pay_3_4,pay_3_5,pay_3_6,pay_3_7,pay_3_8,pay_4_-2,pay_4_0,pay_4_1,pay_4_2,pay_4_3,pay_4_4,pay_4_5,pay_4_6,pay_4_7,pay_4_8,pay_5_-2,pay_5_0,pay_5_2,pay_5_3,pay_5_4,pay_5_5,pay_5_6,pay_5_7,pay_5_8,pay_6_-2,pay_6_0,pay_6_2,pay_6_3,pay_6_4,pay_6_5,pay_6_6,pay_6_7,pay_6_8
ID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,1,0.195650,0.155100,0.034450,0.000000,0.000000,0.000000,0.195650,0.120650,0.034450,0.000000,0.000000,0.000000,9.903488,3.178054,8.272315,8.040125,6.536692,0.000000,0.000000,0.000000,0.000000,6.536692,0.000000,0.000000,0.000000,0.000000,1,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
2,1,0.022350,0.014375,0.022350,0.027267,0.028792,0.027175,0.022350,0.006042,0.014017,0.018933,0.028792,0.010508,11.695247,3.258097,7.894691,7.453562,7.894691,8.093462,8.147867,8.090096,0.000000,6.908755,6.908755,6.908755,0.000000,7.601402,1,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,0,0.324878,0.155856,0.150656,0.159233,0.166089,0.172767,0.308011,0.139189,0.139544,0.148122,0.154978,0.117211,11.407565,3.526361,10.283293,9.548811,9.514880,9.570250,9.612400,9.651816,7.325808,7.313887,6.908755,6.908755,6.908755,8.517393,1,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
4,0,0.939800,0.964660,0.985820,0.566280,0.579180,0.590940,0.899800,0.924280,0.961820,0.544280,0.557800,0.570940,10.819778,3.610918,10.757711,10.783819,10.805517,10.251147,10.273671,10.293771,7.601402,7.610853,7.090910,7.003974,6.975414,6.908755,1,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0
5,0,0.172340,0.113400,0.716700,0.418800,0.382920,0.382620,0.132340,-0.620220,0.516700,0.238800,0.369140,0.369040,10.819778,4.043051,9.061608,8.643121,10.486708,9.949464,9.859901,9.859118,7.601402,10.510041,9.210440,9.105091,6.536692,6.522093,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0


### Train test split

In [5]:
# Extract X and y from df
X = df.drop('y', axis=1).values
#y = df[['y']].values
y = df['y'].values

# Train test split
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.33, random_state=42)

In [6]:
# Define a function`namestr` to access the name of a variable
def namestr(obj, namespace):
    return [name for name in namespace if namespace[name] is obj][0]

In [7]:
# Print the shape of X, y, X_train, X_test, y_train, and y_test
for var in [X, y, X_train, X_test, y_train, y_test]:
    print(namestr(var, globals()), 
          'shape:\t',
          var.shape)

X shape:	 (30000, 91)
y shape:	 (30000,)
X_train shape:	 (20100, 91)
X_test shape:	 (9900, 91)
y_train shape:	 (20100,)
y_test shape:	 (9900,)


### Make pipeline

In [8]:
df_X = df.drop('y', axis=1)

In [9]:
def create_binary_feature_list(df=df_X,  
                               return_binary_features=True):
    """
    Docstring ...
    """
    # Create boolean maskDrop the column with the target values
    binary_mask = df.isin([0, 1]).all()
    
    # If return_binary_features=True,
    # create a list of the binary features.
    # If return_binary_features=False,
    # create a list of the nonbinary features.    
    features_list = list(binary_mask[binary_mask == \
                                     return_binary_features].index)

    return features_list

In [10]:
def binary_feature_index_list(df=df_X, 
                              features_list=None):
    """
    Docstring ...
    """
    
    feature_index_list = [df.columns.get_loc(c) for c  \
                          in df.columns if c in features_list]    
    
    return feature_index_list

In [11]:
binary_features = create_binary_feature_list(df=df_X, 
                                             return_binary_features=True)

non_binary_features = create_binary_feature_list(df=df_X,  
                                                 return_binary_features=False)

binary_index_list = \
    binary_feature_index_list(df=df_X, 
                              features_list=binary_features)

non_binary_index_list = \
    binary_feature_index_list(df=df_X, 
                              features_list=non_binary_features)

In [12]:
print('Binary features:\n')
print(''.join('{:2s}: {:40s}'.format(str(i), col) \
              for i, col in zip(binary_index_list,
                                binary_features)))
print('\n')

print('Non-binary features:\n')
print(''.join('{:2s}: {:40s}'.format(str(i), col) \
              for i, col in zip(non_binary_index_list,
                                non_binary_features)))

Binary features:

26: sex_2                                   27: edu_2                                   28: edu_3                                   29: edu_4                                   30: marriage_1                              31: marriage_2                              32: marriage_3                              33: pay_1_-2                                34: pay_1_0                                 35: pay_1_1                                 36: pay_1_2                                 37: pay_1_3                                 38: pay_1_4                                 39: pay_1_5                                 40: pay_1_6                                 41: pay_1_7                                 42: pay_1_8                                 43: pay_2_-2                                44: pay_2_0                                 45: pay_2_1                                 46: pay_2_2                                 47: pay_2_3                                 48: pay_2_4   

#### User defined preprocessors

In [13]:
class NonBinary_PCA(BaseEstimator, TransformerMixin): 
    def __init__(self):
        self.scaler = PCA(n_components=None, random_state=42)

    # Fit PCA only on the non-binary features
    def fit(self, X, y):
        self.scaler.fit(X[:, non_binary_index_list], y)
        return self

    # Transform only the non-binary features with PCA
    def transform(self, X):
        X_non_binary = \
            self.scaler.transform(X[:, non_binary_index_list])

        X_recombined = X_non_binary

        binary_index_list.sort()
        for col in binary_index_list:
            X_recombined = np.insert(X_recombined, 
                                     col,
                                     X[:, col], 
                                     1)
        return X_recombined

In [14]:
class NonBinary_RobustScaler(BaseEstimator, TransformerMixin): 
    def __init__(self):
        self.scaler = RobustScaler()

    # Fit RobustScaler only on the non-binary features
    def fit(self, X, y):
        self.scaler.fit(X[:, non_binary_index_list], y)
        return self

    # Transform only the non-binary features with RobustScaler
    def transform(self, X):
        X_non_binary = \
            self.scaler.transform(X[:, non_binary_index_list])

        X_recombined = X_non_binary

        binary_index_list.sort()
        for col in binary_index_list:
            X_recombined = np.insert(X_recombined, 
                                     col,
                                     X[:, col], 
                                     1)
        return X_recombined

In [15]:
class NonBinary_StandardScaler(BaseEstimator, TransformerMixin): 
    def __init__(self):
        self.scaler = StandardScaler()

    # Fit StandardScaler only on the non-binary features
    def fit(self, X, y):
        self.scaler.fit(X[:, non_binary_index_list], y)
        return self

    # Transform only the non-binary features with StandardScaler
    def transform(self, X):
        X_non_binary = \
            self.scaler.transform(X[:, non_binary_index_list])

        X_recombined = X_non_binary

        binary_index_list.sort()
        for col in binary_index_list:
            X_recombined = np.insert(X_recombined, 
                                     col,
                                     X[:, col], 
                                     1)
        return X_recombined

In [16]:
class NonBinary_MinMaxScaler(BaseEstimator, TransformerMixin): 
    def __init__(self):
        self.scaler = MinMaxScaler()

    # Fit MinMaxScaler only on the non-binary features
    def fit(self, X, y):
        self.scaler.fit(X[:, non_binary_index_list], y)
        return self

    # Transform only the non-binary features with MinMaxScaler
    def transform(self, X):
        X_non_binary = \
            self.scaler.transform(X[:, non_binary_index_list])

        X_recombined = X_non_binary

        binary_index_list.sort()
        for col in binary_index_list:
            X_recombined = np.insert(X_recombined, 
                                     col,
                                     X[:, col], 
                                     1)
        return X_recombined

#### Define the pipeline

In [19]:
pipe = Pipeline([('preprocessing_1', None), 
                 ('preprocessing_2', None), 
                 ('classifier', DummyClassifier(strategy='most_frequent',
                                                random_state=42))])


param_grid = [
    {'classifier': [DummyClassifier(strategy='most_frequent',
                                    random_state=42)],
     'preprocessing_1': [None], 
     'preprocessing_2': [None]},
    
    {'classifier': [LogisticRegression(random_state=42)],
     'preprocessing_1': [None, NonBinary_RobustScaler()], 
     'preprocessing_2': [None, NonBinary_PCA()],
     'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100],
     'classifier__penalty': ['l1','l2']},

    
    {'classifier': [XGBClassifier(objective='binary:logistic')], 
     'preprocessing_1': [None, NonBinary_RobustScaler()], 
     'preprocessing_2': [None, NonBinary_PCA()],
     'classifier__max_depth': [1, 2, 3, 4, 5, 6, 7],
     'classifier__learning_rate': [0.0001, 0.001, 0.01, 0.1, 0.3],
     'classifier__n_estimators': [1000]}]

In [20]:
cores_used = cpu_count() - 1
cores_used

3

In [21]:
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='roc_auc',
                    verbose=100, n_jobs=cores_used)

grid.fit(X_train, y_train)

print("Best params:\n{}\n".format(grid.best_params_))
print("Best cross-validation score: {:.2f}".format(grid.best_score_))

Fitting 5 folds for each of 189 candidates, totalling 945 fits
Memmaping (shape=(20100, 91), dtype=float64) to new file /var/folders/zb/s9ymkj7s5q7b6qrkyq4wm0km0000gn/T/joblib_memmaping_pool_13189_4484218384/13189-4654898480-2fc6e168b2a1757b91a3e99cfd5ce9df.pkl
Pickling array (shape=(20100,), dtype=int64).
Pickling array (shape=(16079,), dtype=int64).
Pickling array (shape=(4021,), dtype=int64).
Memmaping (shape=(20100, 91), dtype=float64) to old file /var/folders/zb/s9ymkj7s5q7b6qrkyq4wm0km0000gn/T/joblib_memmaping_pool_13189_4484218384/13189-4654898480-2fc6e168b2a1757b91a3e99cfd5ce9df.pkl
Pickling array (shape=(20100,), dtype=int64).
Pickling array (shape=(16079,), dtype=int64).
Pickling array (shape=(4021,), dtype=int64).
[CV] classifier=DummyClassifier(constant=None, random_state=42, strategy='most_frequent'), preprocessing_1=None, preprocessing_2=None 
[CV] classifier=DummyClassifier(constant=None, random_state=42, strategy='most_frequent'), preprocessing_1=None, preprocessing_2=N

[CV] classifier=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False), classifier__C=0.001, classifier__penalty=l1, preprocessing_1=None, preprocessing_2=NonBinary_PCA() 
[CV]  classifier=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False), classifier__C=0.001, classifier__penalty=l1, preprocessing_1=None, preprocessing_2=None, score=0.697298085107905, total=   0.5s
[CV] classifier=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, sol

          verbose=0, warm_start=False), classifier__C=0.001, classifier__penalty=l1, preprocessing_1=NonBinary_RobustScaler(), preprocessing_2=None 
[Parallel(n_jobs=3)]: Done  15 tasks      | elapsed:    6.0s
Memmaping (shape=(20100, 91), dtype=float64) to old file /var/folders/zb/s9ymkj7s5q7b6qrkyq4wm0km0000gn/T/joblib_memmaping_pool_13189_4484218384/13189-4654898480-2fc6e168b2a1757b91a3e99cfd5ce9df.pkl
Pickling array (shape=(20100,), dtype=int64).
Pickling array (shape=(16081,), dtype=int64).
Pickling array (shape=(4019,), dtype=int64).
[CV]  classifier=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False), classifier__C=0.001, classifier__penalty=l1, preprocessing_1=NonBinary_RobustScaler(), preprocessing_2=None, score=0.6528608928571429, total=   1.1s
[CV] classifier=Logistic

          verbose=0, warm_start=False), classifier__C=0.001, classifier__penalty=l1, preprocessing_1=NonBinary_RobustScaler(), preprocessing_2=NonBinary_PCA() 
[Parallel(n_jobs=3)]: Done  21 tasks      | elapsed:   12.2s
[Parallel(n_jobs=3)]: Done  22 tasks      | elapsed:   12.3s
Memmaping (shape=(20100, 91), dtype=float64) to old file /var/folders/zb/s9ymkj7s5q7b6qrkyq4wm0km0000gn/T/joblib_memmaping_pool_13189_4484218384/13189-4654898480-2fc6e168b2a1757b91a3e99cfd5ce9df.pkl
Pickling array (shape=(20100,), dtype=int64).
Pickling array (shape=(16081,), dtype=int64).
Pickling array (shape=(4019,), dtype=int64).
Memmaping (shape=(20100, 91), dtype=float64) to old file /var/folders/zb/s9ymkj7s5q7b6qrkyq4wm0km0000gn/T/joblib_memmaping_pool_13189_4484218384/13189-4654898480-2fc6e168b2a1757b91a3e99cfd5ce9df.pkl
Pickling array (shape=(20100,), dtype=int64).
Pickling array (shape=(16079,), dtype=int64).
Pickling array (shape=(4021,), dtype=int64).
[CV]  classifier=LogisticRegression(C=1.0, cla

          verbose=0, warm_start=False), classifier__C=0.001, classifier__penalty=l1, preprocessing_1=NonBinary_RobustScaler(), preprocessing_2=NonBinary_PCA(), score=0.7223660398143047, total=   2.1s[CV]  classifier=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False), classifier__C=0.001, classifier__penalty=l1, preprocessing_1=NonBinary_RobustScaler(), preprocessing_2=NonBinary_PCA(), score=0.7140296067926094, total=   2.1s

[CV] classifier=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=42, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False), classifier__C=0.001, classifier__penalty=l2, preprocessing_1=None, preprocessin

...

       scale_pos_weight=1, seed=0, silent=True, subsample=1), classifier__learning_rate=0.3, classifier__max_depth=6, classifier__n_estimators=1000, preprocessing_1=NonBinary_RobustScaler(), preprocessing_2=NonBinary_PCA() 
[Parallel(n_jobs=3)]: Done 918 tasks      | elapsed: 299.9min
Memmaping (shape=(20100, 91), dtype=float64) to old file /var/folders/zb/s9ymkj7s5q7b6qrkyq4wm0km0000gn/T/joblib_memmaping_pool_13189_4484218384/13189-4654898480-2fc6e168b2a1757b91a3e99cfd5ce9df.pkl
Pickling array (shape=(20100,), dtype=int64).
Pickling array (shape=(16079,), dtype=int64).
Pickling array (shape=(4021,), dtype=int64).
[CV]  classifier=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=nan, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1), classifier__learning_rate=0.3

       scale_pos_weight=1, seed=0, silent=True, subsample=1), classifier__learning_rate=0.3, classifier__max_depth=6, classifier__n_estimators=1000, preprocessing_1=NonBinary_RobustScaler(), preprocessing_2=NonBinary_PCA(), score=0.7334217340610447, total= 1.9min
[CV] classifier=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=nan, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1), classifier__learning_rate=0.3, classifier__max_depth=7, classifier__n_estimators=1000, preprocessing_1=None, preprocessing_2=None 
[Parallel(n_jobs=3)]: Done 924 tasks      | elapsed: 303.8min
Memmaping (shape=(20100, 91), dtype=float64) to old file /var/folders/zb/s9ymkj7s5q7b6qrkyq4wm0km0000gn/T/joblib_memmaping_pool_13189_4484218384/13189-4654898480-2fc6e168b2a1757b91a3e99cfd5ce9df.

       scale_pos_weight=1, seed=0, silent=True, subsample=1), classifier__learning_rate=0.3, classifier__max_depth=7, classifier__n_estimators=1000, preprocessing_1=None, preprocessing_2=None, score=0.7330900077969084, total= 1.9min
[CV] classifier=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=nan, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1), classifier__learning_rate=0.3, classifier__max_depth=7, classifier__n_estimators=1000, preprocessing_1=None, preprocessing_2=NonBinary_PCA() 
[Parallel(n_jobs=3)]: Done 930 tasks      | elapsed: 307.7min
Memmaping (shape=(20100, 91), dtype=float64) to old file /var/folders/zb/s9ymkj7s5q7b6qrkyq4wm0km0000gn/T/joblib_memmaping_pool_13189_4484218384/13189-4654898480-2fc6e168b2a1757b91a3e99cfd5ce9df.pkl
Pickling array (

[CV]  classifier=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=nan, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1), classifier__learning_rate=0.3, classifier__max_depth=7, classifier__n_estimators=1000, preprocessing_1=NonBinary_RobustScaler(), preprocessing_2=None, score=0.7414098214285715, total= 2.0min
[CV] classifier=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=nan, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1), classifier__learning_rate=0.3, classifier__max_depth=7, classifier__n_estimators=1000, preprocessing_1=NonBinary

       scale_pos_weight=1, seed=0, silent=True, subsample=1), classifier__learning_rate=0.3, classifier__max_depth=7, classifier__n_estimators=1000, preprocessing_1=NonBinary_RobustScaler(), preprocessing_2=NonBinary_PCA() 
Memmaping (shape=(20100, 91), dtype=float64) to old file /var/folders/zb/s9ymkj7s5q7b6qrkyq4wm0km0000gn/T/joblib_memmaping_pool_13189_4484218384/13189-4654898480-2fc6e168b2a1757b91a3e99cfd5ce9df.pkl
Pickling array (shape=(20100,), dtype=int64).
Pickling array (shape=(16081,), dtype=int64).
Pickling array (shape=(4019,), dtype=int64).
[CV]  classifier=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=nan, n_estimators=100, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1), classifier__learning_rate=0.3, classifier__max_depth=7, classifier__n_estimators=1000, prep

#### Save the grid search object as a pickle file

In [22]:
joblib.dump(grid.best_estimator_, 'pipe_pickle_20171023.pkl')

['pipe_pickle_20171023.pkl']

In [23]:
joblib.dump(grid, 'pipe_pickle_all_20171023.pkl')

['pipe_pickle_all_20171023.pkl']

In [50]:
clf = joblib.load('pipe_pickle_20171023.pkl') 

In [51]:
clf

Pipeline(memory=None,
     steps=[('preprocessing_1', None), ('preprocessing_2', None), ('classifier', XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=4,
       min_child_weight=1, missing=nan, n_estimators=1000, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1))])

In [52]:
loaded_gs = joblib.load('pipe_pickle_all_20171023.pkl') 

In [53]:
loaded_gs.best_estimator_

Pipeline(memory=None,
     steps=[('preprocessing_1', None), ('preprocessing_2', None), ('classifier', XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=4,
       min_child_weight=1, missing=nan, n_estimators=1000, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1))])

In [24]:
grid.best_estimator_

Pipeline(memory=None,
     steps=[('preprocessing_1', None), ('preprocessing_2', None), ('classifier', XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=4,
       min_child_weight=1, missing=None, n_estimators=1000, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1))])

In [25]:
grid.best_score_

0.78500958655627306

In [56]:
1 - accuracy_score(y_test, grid.best_estimator_.predict(X_test))

0.17909090909090908

In [58]:
1 - accuracy_score(y_train, grid.best_estimator_.predict(X_train))

0.16786069651741298

In [60]:
baseline_mf = DummyClassifier(strategy='most_frequent', 
                              random_state=42)
baseline_mf.fit(X_train, y_train)

DummyClassifier(constant=None, random_state=42, strategy='most_frequent')

In [61]:
1 - accuracy_score(y_train, baseline_mf.predict(X_train))

0.2227860696517413

In [62]:
1 - accuracy_score(y_test, baseline_mf.predict(X_test))

0.21797979797979794

In [33]:
roc_auc_score(y_test, grid.best_estimator_.predict(X_test))

0.65370866850746578

In [31]:
roc_auc_score(y_train, grid.best_estimator_.predict(X_train))

0.67408655548064422

In [34]:
roc_auc_score(y_train, grid.predict(X_train))

0.67408655548064422

In [35]:
clf = XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=4,
       min_child_weight=1, missing=None, n_estimators=1000, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [36]:
clf.fit(X_train, y_train)

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.01, max_delta_step=0, max_depth=4,
       min_child_weight=1, missing=None, n_estimators=1000, nthread=-1,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [37]:
roc_auc_score(y_train, clf.predict(X_train))

0.67408655548064422

In [38]:
roc_auc_score(y_test, clf.predict(X_test))

0.65370866850746578

In [46]:
results_df = pd.DataFrame(grid.cv_results_)

In [49]:
results_df.sort_values('mean_test_score', ascending=False)

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_classifier,param_classifier__C,param_classifier__learning_rate,param_classifier__max_depth,param_classifier__n_estimators,param_classifier__penalty,param_preprocessing_1,param_preprocessing_2,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,split3_test_score,split3_train_score,split4_test_score,split4_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
119,79.254328,0.691932,0.785010,0.835458,"XGBClassifier(base_score=0.5, colsample_byleve...",NaN,0.01,4,1000,NaN,NonBinary_RobustScaler(),None,"{'classifier': XGBClassifier(base_score=0.5, c...",1,0.792856,0.835711,0.786357,0.834962,0.793849,0.834653,0.778440,0.834422,0.773539,0.837544,0.998259,0.078784,0.007953,1.129890e-03
117,72.819231,0.270420,0.785010,0.835458,"XGBClassifier(base_score=0.5, colsample_byleve...",NaN,0.01,4,1000,NaN,None,None,"{'classifier': XGBClassifier(base_score=0.5, c...",1,0.792856,0.835711,0.786357,0.834962,0.793849,0.834653,0.778440,0.834422,0.773539,0.837544,4.385856,0.037455,0.007953,1.129890e-03
113,52.225261,0.153847,0.784536,0.813992,"XGBClassifier(base_score=0.5, colsample_byleve...",NaN,0.01,3,1000,NaN,None,None,"{'classifier': XGBClassifier(base_score=0.5, c...",3,0.792972,0.814087,0.784674,0.813956,0.795258,0.811498,0.776212,0.814728,0.773559,0.815692,0.377969,0.010454,0.008671,1.390191e-03
115,52.295308,0.464690,0.784536,0.813992,"XGBClassifier(base_score=0.5, colsample_byleve...",NaN,0.01,3,1000,NaN,NonBinary_RobustScaler(),None,"{'classifier': XGBClassifier(base_score=0.5, c...",3,0.792972,0.814087,0.784674,0.813956,0.795258,0.811498,0.776212,0.814728,0.773559,0.815692,0.321389,0.023746,0.008671,1.390191e-03
121,86.384442,0.255856,0.784050,0.861208,"XGBClassifier(base_score=0.5, colsample_byleve...",NaN,0.01,5,1000,NaN,None,None,"{'classifier': XGBClassifier(base_score=0.5, c...",5,0.790865,0.863208,0.786833,0.858237,0.793470,0.862625,0.778048,0.858780,0.771027,0.863191,0.500828,0.014301,0.008349,2.220667e-03
123,88.376608,0.628739,0.784050,0.861208,"XGBClassifier(base_score=0.5, colsample_byleve...",NaN,0.01,5,1000,NaN,NonBinary_RobustScaler(),None,"{'classifier': XGBClassifier(base_score=0.5, c...",5,0.790865,0.863208,0.786833,0.858237,0.793470,0.862625,0.778048,0.858780,0.771027,0.863191,1.704908,0.029859,0.008349,2.220667e-03
116,59.497859,0.831051,0.783968,0.818673,"XGBClassifier(base_score=0.5, colsample_byleve...",NaN,0.01,3,1000,NaN,NonBinary_RobustScaler(),NonBinary_PCA(),"{'classifier': XGBClassifier(base_score=0.5, c...",7,0.789868,0.817001,0.786354,0.816020,0.793221,0.819437,0.779191,0.820948,0.771199,0.819959,0.185499,0.031342,0.007899,1.857533e-03
120,80.709365,0.923457,0.783438,0.842375,"XGBClassifier(base_score=0.5, colsample_byleve...",NaN,0.01,4,1000,NaN,NonBinary_RobustScaler(),NonBinary_PCA(),"{'classifier': XGBClassifier(base_score=0.5, c...",8,0.789462,0.838597,0.786967,0.840751,0.789850,0.844046,0.780657,0.845363,0.770250,0.843117,3.018522,0.018948,0.007369,2.415895e-03
112,47.381702,0.928132,0.783059,0.799231,"XGBClassifier(base_score=0.5, colsample_byleve...",NaN,0.01,2,1000,NaN,NonBinary_RobustScaler(),NonBinary_PCA(),"{'classifier': XGBClassifier(base_score=0.5, c...",9,0.787580,0.798525,0.784894,0.797486,0.793679,0.797544,0.777584,0.800960,0.771555,0.801641,0.821109,0.086698,0.007731,1.742907e-03
109,39.979815,0.183746,0.782641,0.798127,"XGBClassifier(base_score=0.5, colsample_byleve...",NaN,0.01,2,1000,NaN,None,None,"{'classifier': XGBClassifier(base_score=0.5, c...",10,0.790138,0.797091,0.781544,0.797958,0.795340,0.795771,0.775620,0.799191,0.770556,0.800622,0.506184,0.021551,0.009099,1.673969e-03


In [43]:
type(grid.grid_scores_)

/anaconda/lib/python3.6/site-packages/sklearn/model_selection/_search.py:747: DeprecationWarning: The grid_scores_ attribute was deprecated in version 0.18 in favor of the more elaborate cv_results_ attribute. The grid_scores_ attribute will not be available from 0.20
  DeprecationWarning)


list

In [32]:
grid

GridSearchCV(cv=5, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('preprocessing_1', None), ('preprocessing_2', None), ('classifier', DummyClassifier(constant=None, random_state=42, strategy='most_frequent'))]),
       fit_params=None, iid=True, n_jobs=3,
       param_grid=[{'classifier': [DummyClassifier(constant=None, random_state=42, strategy='most_frequent')], 'preprocessing_1': [None], 'preprocessing_2': [None]}, {'classifier': [LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, mul..., 'classifier__learning_rate': [0.0001, 0.001, 0.01, 0.1, 0.3], 'classifier__n_estimators': [1000]}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='roc_auc', verbose=100)

# ....

In [ ]:
n_pca = NonBinary_PCA()
n_rsl = NonBinary_RobustScaler()

X_train_p = n_rsl.fit_transform(X_train, y_train)
X_train_p = n_pca.fit_transform(X_train_p, y_train)

In [ ]:
rfe = RFE(XGBClassifier(objective='binary:logistic', 
                        max_depth=4, 
                        learning_rate=0.01,
                        n_estimators=1000), 
          n_features_to_select=50, 
          step=1,
          verbose=100)

rfe.fit(X_train_p, y_train)

In [ ]:
X_train_rfe_ed = rfe.transform(X_train_p)

In [ ]:
X_train_rfe_ed.shape

In [ ]:
pipe = Pipeline([('classifier', DummyClassifier(strategy='most_frequent',
                                                random_state=42))])


param_grid = [
    {'classifier': [DummyClassifier(strategy='most_frequent',
                                    random_state=42)]},
    
    {'classifier': [XGBClassifier(objective='binary:logistic')], 
     'classifier__max_depth': [4],
     'classifier__learning_rate': [0.01],
     'classifier__n_estimators': [1000]}]

#NonBinary_RobustScaler
#NonBinary_PCA

In [ ]:
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='roc_auc',
                    verbose=100, n_jobs=cores_used)

#grid.fit(X_train, y_train)
grid.fit(X_train_rfe_ed, y_train)

print("Best params:\n{}\n".format(grid.best_params_))
print("Best cross-validation score: {:.2f}".format(grid.best_score_))

In [ ]:
X_train[:, non_binary_index_list]

In [ ]:
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='roc_auc',
                    verbose=100, n_jobs=cores_used)

#grid.fit(X_train, y_train)
grid.fit(X_train[:, non_binary_index_list], y_train)

print("Best params:\n{}\n".format(grid.best_params_))
print("Best cross-validation score: {:.2f}".format(grid.best_score_))

In [ ]:
print(''.join('{:2s}: {:40s}'.format(str(i), col) \
              for i, col in zip(rfe.ranking_,
                                df_X.columns)))
print('\n')

In [ ]:
XGBClassifier(objective='binary:logistic', 
              max_depth=4, 
              learning_rate=0.01,
              n_estimators=1000)
     'preprocessing_1': [NonBinary_RobustScaler()], 
     'preprocessing_2': [NonBinary_PCA()],


In [ ]:
   {'classifier': [XGBClassifier(objective='binary:logistic')], 
     'preprocessing_1': [NonBinary_RobustScaler()], 
     'preprocessing_2': [NonBinary_PCA()],
     'classifier__max_depth': [4],
     'classifier__learning_rate': [0.01],
     'classifier__n_estimators': [1000]}]

In [ ]:
X_train.shape

In [ ]:
interactions = PolynomialFeatures(degree=2, 
                                  interaction_only=True, 
                                  include_bias=False)

In [ ]:
pipe = Pipeline([('preprocessing_1', None), 
                 ('preprocessing_2', None), 
                 ('classifier', DummyClassifier(strategy='most_frequent',
                                                random_state=42))])


param_grid = [
    {'classifier': [DummyClassifier(strategy='most_frequent',
                                    random_state=42)],
     'preprocessing_1': [None], 
     'preprocessing_2': [None]},
    
    {'classifier': [XGBClassifier(objective='binary:logistic')], 
     'preprocessing_1': [None], 
     'preprocessing_2': [NonBinary_RobustScaler],
     'classifier__max_depth': [4],
     'classifier__learning_rate': [0.01],
     'classifier__n_estimators': [1000]}]

#NonBinary_RobustScaler
#NonBinary_PCA

In [ ]:
pipe = Pipeline([('preprocessing_1', None), 
                 ('classifier', DummyClassifier(strategy='most_frequent',
                                                random_state=42))])


param_grid = [
    {'classifier': [DummyClassifier(strategy='most_frequent',
                                    random_state=42)],
     'preprocessing_1': [None]},
    
    {'classifier': [XGBClassifier(objective='binary:logistic')], 
     'preprocessing_1': [NonBinary_RobustScaler],
     'classifier__max_depth': [4],
     'classifier__learning_rate': [0.01],
     'classifier__n_estimators': [1000]}]

#NonBinary_RobustScaler
#NonBinary_PCA

In [ ]:
# verbose=100, 
# n_jobs=cores_used)

grid = GridSearchCV(pipe, param_grid, cv=5, 
                    scoring='roc_auc',
                    verbose=100, 
                    n_jobs=cores_used)
# ...
grid.fit(X_train, y_train)

print("Best params:\n{}\n".format(grid.best_params_))
print("Best cross-validation score: {:.2f}".format(grid.best_score_))
#print("Test-set score: {:.2f}".format(grid.score(X_test, y_test)))

In [ ]:
grid.best_score_

In [59]:
baseline_mf = DummyClassifier(strategy='most_frequent', 
                              random_state=42)

param_grid = {'strategy': ['most_frequent', 'stratified']}

grid = GridSearchCV(baseline_mf, param_grid, cv=5, 
                    scoring='roc_auc',
                    verbose=0, 
                    n_jobs=cores_used)
# ...
grid.fit(X_train, y_train)

print("Best params:\n{}\n".format(grid.best_params_))
print("Best cross-validation score: {:.2f}".format(grid.best_score_))

Best params:
{'strategy': 'stratified'}

Best cross-validation score: 0.50


In [ ]:
plt.scatter(pcs[:,0], pcs[:,1], c=-y_train, s=5)

In [ ]:
plt.scatter(pcs[:,0], pcs[:,2], c=-y_train, s=5)

In [ ]:
plt.scatter(pcs[:,0], pcs[:,3], c=-y_train, s=5)

In [ ]:
plt.scatter(pcs[:,0], pcs[:,4], c=-y_train, s=5)

In [ ]:
plt.scatter(pcs[:,1], pcs[:,2], c=-y_train, s=5)

In [ ]:
plt.scatter(pcs[:,1], pcs[:,3], c=-y_train, s=5)

In [ ]:
pcs_all = pca.fit_transform(rbsl.fit_transform(X_train))

In [ ]:
plt.scatter(pcs_all[:,0], pcs_all[:,1], c=-y_train, s=5)

In [ ]:
plt.scatter(pcs_all[:,1], pcs_all[:,2], c=-y_train, s=5)

In [ ]:
pipe = Pipeline([('preprocessing', NonBinary_RobustScaler()), 
                 ('classifier', LogisticRegression())])

param_grid = [
    {'preprocessing': [NonBinary_RobustScaler(), 
                       NonBinary_StandardScaler(),
                       NonBinary_MinMaxScaler()],
     'classifier': [LogisticRegression()], 
     'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100],
     'classifier__penalty': ['l1','l2']}]

#     'classifier__C': [0.001, 0.01, 0.1, 1, 10, 100, 1000],

In [ ]:
grid.best_estimator_

In [ ]:
from sklearn.externals import joblib

In [ ]:
#joblib.dump(grid.best_estimator_, 'pipe_pickle_20171019.pkl')

In [ ]:
clf = joblib.load('pipe_pickle_20171019.pkl') 

In [ ]:
clf

In [ ]:
clf.predict(X_test)

In [ ]:
clf.score(X_train, y_train)

In [ ]:
clf.score(X_test, y_test)